In [1]:
import tqdm
import glob
import cv2
import dlib
import numpy as np

from collections import OrderedDict

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
SOURCE_PATH = "dgw/input"
RESULT_PATH = "dgw/preprocessed"

FACE_DETECTOR_MODEL = "pre_trained_models/mmod_human_face_detector.dat"

In [ ]:
def get_facial_landmarks(img):
    import face_alignment
    
    face_detector = face_alignment.FaceAlignment(
        face_alignment.LandmarksType._2D,
        flip_input=False,
        device='cpu'
    )

    facial_landmarks = face_detector.get_landmarks(img)
    
    return facial_landmarks


In [ ]:
class Point:
    def __init__(self, X, Y):
        self.x = X
        self.y = Y

class C_Rectangle:
    def __init__(self, topLeft, bottomRight):
        self.topLeft = topLeft
        self.bottomRight = bottomRight
        
        self.height = topLeft.y - bottomRight.y
        self.width = bottomRight.x - topLeft.x
    
    def get_area(self):
        area = self.height * self.width
        return area

In [ ]:
def get_driver_face(img):
    upsample_num = 1

    hog_face_detector = dlib.get_frontal_face_detector()
    cnn_face_detector = dlib.cnn_face_detection_model_v1(FACE_DETECTOR_MODEL)
    
    detected_faces = hog_face_detector(img, 1)
    is_using_cnn = False
    if not detected_faces:
        detected_faces = cnn_face_detector(img, upsample_num)
        is_using_cnn = True

    if not detected_faces:
        print("Unable to detect any faces at {}".format("ABC"))
    
    main_face = None
    main_face_area = 0
    for i in range(len(detected_faces)):
        current_face = detected_faces[i]
        if is_using_cnn:
            current_face = detected_faces[i].rect
        
        top_left = Point(current_face.left(), current_face.top())
        bottom_right = Point(current_face.right(), current_face.bottom())
    
        current_face_area = C_Rectangle(top_left, bottom_right).get_area()
        if not main_face or current_face_area > main_face_area:
            main_face = current_face
            main_face_area = current_face_area

    return img[main_face.top():main_face.bottom(), main_face.left():main_face.right()]

In [ ]:
def get_driver_eyes(landmarks):
    
    FACIAL_LANDMARKS_IDXS = OrderedDict([
        ("mouth", (48, 68)),
        ("right_eyebrow", (17, 22)),
        ("left_eyebrow", (22, 27)),
        ("right_eye", (36, 42)),
        ("left_eye", (42, 48)),
        ("nose", (27, 35)),
        ("jaw", (0, 17))
    ])
    
    left_eye = landmarks[FACIAL_LANDMARKS_IDXS["left_eye"][0]:FACIAL_LANDMARKS_IDXS["left_eye"][1]]
    right_eye = landmarks[FACIAL_LANDMARKS_IDXS["right_eye"][0]:FACIAL_LANDMARKS_IDXS["right_eye"][1]]
    
    eyes = np.concatenate((left_eye, right_eye))
    top_left_x = min(eyes, key = lambda t: t[0])[0]
    top_left_y = min(eyes, key = lambda t: t[1])[1]
    
    bottom_right_x = max(eyes, key = lambda t: t[0])[0]
    bottom_right_y = max(eyes, key = lambda t: t[1])[1]
    
    return ((top_left_x, top_left_y), (bottom_right_x, bottom_right_y))

In [ ]:
for file_path in glob.glob("{src}/[0-9]/*.png".format(src=SOURCE_PATH)):
    path_info = file_path.split("/")
    IMAGE_NAME = path_info[-1].replace(".png", "")
    ZONE_NAME = path_info[-2]
    
    img = dlib.load_rgb_image(file_path)
    
    face_img = get_driver_face(img)
    op_path = "{dir}/{zone}/cropped_{img}.png".format(dir=RESULT_PATH, zone=ZONE_NAME, img=IMAGE_NAME)
#     print(op_path)
    cv2.imwrite(op_path, face_img)

    landmarks = get_facial_landmarks(face_img)    
#     for i in range(len(landmarks[0])):
#         print(i, landmarks[0][i])

    eyes_boundng_box = get_driver_eyes(landmarks[0])
    eye_image = cv2.rectangle(face_img, eyes_boundng_box[0], eyes_boundng_box[1], (255, 0, 0) , 2) 
    op_path = "{dir}/{zone}/eye_{img}.png".format(dir=RESULT_PATH, zone=ZONE_NAME, img=IMAGE_NAME)
    print(op_path)
    cv2.imwrite(op_path, eye_image)
    

    